In [1]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

UnicodeDecodeError: 'gbk' codec can't decode byte 0xba in position 2367: illegal multibyte sequence

In [ ]:
import math
import numpy as np
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy.stats import poisson
from patsy import dmatrices
import statsmodels.graphics.tsaplots as tsa
from matplotlib import pyplot as plt
from statsmodels.tools.numdiff import approx_hess1, approx_hess2, approx_hess3
import pandas as pd

df = pd.read_csv('D:/data/Zircons/Roberts_Spencer_2015.csv')
ages = df["U-Pb (Ma)"]
data = plt.hist(ages, bins = 200)

In [ ]:
import pymc as pm

print(f"Running on PyMC v{pm.__version__}")

In [ ]:
zircons_data = data[0]
ages = data[1][1:]
print(len(ages))
print(max(ages))

In [ ]:
plt.plot(ages, zircons_data, "o", markersize=3, alpha=1)
plt.ylabel("Zircons count")
plt.xlabel("Ages");

In [ ]:
tr =pm.distributions.transforms

Order = tr.Ordered()

#Logodd = tr.LogOddsTransform()

#chain_tran = tr.Chain([Logodd, Order])

k = 5

In [ ]:
with pm.Model() as model:
    
    k =8
    
    rates = pm.Exponential("rates",1.0,shape = k + 1)
    
    switchpoints = pm.Uniform("switchpoints", lower = 0, upper = max(ages) , shape = k, transform = Order,  initval=np.linspace(100,max(ages)-100,k))
    #x = pm.Uniform("x", 0.0, 1.0, shape = n, transform = chain_tran)
    rate = rates[0]
    for i in range(k):
        rate = pm.math.switch(switchpoints[i] >= ages, rate, rates[i+1])
        
    disasters = pm.Poisson("counts", rate, observed=zircons_data)
    
    tr100000 = pm.sample(draws =80000,tune = 20000, cores = 1)

In [ ]:
with model:
    az.plot_trace(tr100000)
    
pm.summary(tr100000)

In [ ]:
with pm.Model() as model:
    
    k =8
    
    rates = pm.Exponential("rates",1.0,shape = k + 1)
    
    switchpoints = pm.Uniform("switchpoints", lower = 0, upper = max(ages) , shape = k, transform = Order,  initval=np.linspace(100,max(ages)-100,k))
    #x = pm.Uniform("x", 0.0, 1.0, shape = n, transform = chain_tran)
    rate = rates[0]
    for i in range(k):
        rate = pm.math.switch(switchpoints[i] >= ages, rate, rates[i+1])
        
    disasters = pm.Poisson("counts", rate, observed=zircons_data)
    
    tr150000 = pm.sample(draws =1000,tune = 1000, cores = 4)

In [ ]:
with model:
    az.plot_trace(tr150000)
    
pm.summary(tr150000)